In [1]:
import pickle
# import omegaconf
import warnings
import pickle
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_name = 'full_fine_tuning_50epochs_paper_final2'

datas=( 'uc-merced-land-use-dataset',
       'stanford_cars', 
       'caltech101', 
       'fgvc-aircraft-2013b', 
       'flowers-102',
         'oxford-iiit-pet'  ) #'dtd',

losses=( 'TRADES_v2', 'CLASSIC_AT' ) # 

model_parameters = {
    'convnext_base': 86.0,
    'convnext_tiny': 28.0,
    'deit_small': 22.0,
    'vit_base': 86.0,
    'vit_small': 22.0,
    'resnet50': 25.0,
    'eva02_base': 78.0,
    'eva02_tiny': 24.0,
    'swin_base': 88.0,
    'swin_tiny': 28.0,
    'coatnet_0': 33.0,
    'coatnet_2': 77.0
}

# Two backbone groups
backbones=(
  'CLIP-convnext_base_w-laion_aesthetic-s13B-b82K',
  'CLIP-convnext_base_w-laion2B-s13B-b82K',
  'deit_small_patch16_224.fb_in1k',
  'robust_resnet50',
  'vit_small_patch16_224.augreg_in21k',
  'convnext_base.fb_in1k',
  'resnet50.a1_in1k',
  'robust_vit_base_patch16_224',
  'vit_base_patch16_224.mae',
  'vit_small_patch16_224.dino',
  'convnext_base.fb_in22k',

  'robust_convnext_base',
  'vit_base_patch16_224.augreg_in1k',
  'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_clip_224.laion2b',
  'convnext_tiny.fb_in1k',
  'robust_convnext_tiny',
  'robust_deit_small_patch16_224',
  'vit_small_patch16_224.augreg_in1k',
  'convnext_tiny.fb_in22k',
  'vit_base_patch16_clip_224.laion2b_ft_in1k',
  'vit_base_patch16_224.augreg_in21k_ft_in1k',

  'vit_small_patch16_224.augreg_in21k_ft_in1k',
  'eva02_base_patch14_224.mim_in22k',
  'eva02_tiny_patch14_224.mim_in22k',
  'swin_base_patch4_window7_224.ms_in22k_ft_in1k',
  'swin_tiny_patch4_window7_224.ms_in1k',
  'convnext_base.clip_laion2b_augreg_ft_in12k_in1k',
  'convnext_base.fb_in22k_ft_in1k',
  'convnext_tiny.fb_in22k_ft_in1k',
  'coatnet_0_rw_224.sw_in1k',
  'coatnet_2_rw_224.sw_in12k_ft_in1k',
  'coatnet_2_rw_224.sw_in12k',
)

import math
from typing import Mapping, Tuple

def sums_from_dict(scores):

    values = []
    for v in scores.values():
        if v is None or (isinstance(v, float) and math.isnan(v)):
            return math.nan, math.nan
        values.append(float(v))

    arith_sum = sum(values)
    geom_sum  = math.prod(values)            # built‑in in Python ≥3.8
    return arith_sum, geom_sum

final_data = []


for loss in losses:
    for data in datas:
        for backbone in backbones:

            try:
                name ='{}_{}_{}'.format(backbone, data, loss)
                print( '../results/{}/{}.pkl'.format(project_name, name) )
                with open('../results/{}/{}.pkl'.format(project_name, name), 'rb') as f:
                    result = pickle.load(f)
                                    # result = saved_data[name]["statistics"]

                arith_sum, geom_sum = sums_from_dict(result)
                result['sum'] = arith_sum
                result['geom'] = geom_sum

            except:
                
                result = {'clean_acc': None, 'Linf_acc': None, 'L2_acc': None, 'L1_acc': None, 'common_acc': None, 
                          'sum':None, 'geom':None, }
                
            for key, value in model_parameters.items():
                if key in backbone:  # Match the model name in the backbone string
                    result['model_size'] = 1 if value < 50 else 2
                    break

            result['backbone'] = backbone
            result['dataset'] = data
            result['loss_function'] = loss  
            final_data.append(result)


../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [3]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

# df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

import pandas as pd

df = pd.DataFrame(final_data)

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

df = df[ df.model_size == 2 ]

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

grouped_df.to_csv("./{}.csv".format(project_name))

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

df_sorted

Percentage of NaN values: 18.07%


dataset                                                       caltech101  \
metric                                                            L1_acc   
backbone                                        loss_function              
vit_base_patch16_clip_224.laion2b_ft_in1k       TRADES_v2       0.776000   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   TRADES_v2            NaN   
eva02_base_patch14_224.mim_in22k                TRADES_v2            NaN   
vit_base_patch16_clip_224.laion2b               TRADES_v2       0.757333   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   CLASSIC_AT           NaN   
vit_base_patch16_224.augreg_in1k                TRADES_v2       0.717333   
robust_convnext_base                            TRADES_v2            NaN   
vit_base_patch16_224.augreg_in1k                CLASSIC_AT      0.681000   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k TRADES_v2            NaN   
robust_convnext_base                            CLASSIC_AT           NaN   
convnext_base.fb_in22k_ft_in1k                  TRADES_v2            NaN   
                                                CLASSIC_AT           NaN   
vit_base_patch16_224.augreg_in21k               CLASSIC_AT      0.658667   
vit_base_patch16_clip_224.laion2b               CLASSIC_AT      0.609000   
vit_base_patch16_224.augreg_in21k               TRADES_v2       0.733000   
vit_base_patch16_224.augreg_in21k_ft_in1k       CLASSIC_AT      0.730000   
                                                TRADES_v2       0.666000   
eva02_base_patch14_224.mim_in22k                CLASSIC_AT           NaN   
vit_base_patch16_clip_224.laion2b_ft_in1k       CLASSIC_AT      0.460000   
coatnet_2_rw_224.sw_in12k_ft_in1k               CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
coatnet_2_rw_224.sw_in12k                       TRADES_v2            NaN   
                                                CLASSIC_AT           NaN   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k CLASSIC_AT           NaN   
robust_vit_base_patch16_224                     TRADES_v2       0.755000   
vit_base_patch16_224.mae                        CLASSIC_AT      0.587000   
robust_vit_base_patch16_224                     CLASSIC_AT      0.760000   
vit_base_patch16_224.mae                        TRADES_v2       0.155000   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  CLASSIC_AT           NaN   
convnext_base.fb_in1k                           CLASSIC_AT           NaN   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  TRADES_v2            NaN   
convnext_base.fb_in1k                           TRADES_v2            NaN   
CLIP-convnext_base_w-laion2B-s13B-b82K          TRADES_v2            NaN   
convnext_base.fb_in22k                          TRADES_v2            NaN   
                                                CLASSIC_AT           NaN   
CLIP-convnext_base_w-laion2B-s13B-b82K          CLASSIC_AT           NaN   

dataset                                                                  \
metric                                                           L2_acc   
backbone                                        loss_function             
vit_base_patch16_clip_224.laion2b_ft_in1k       TRADES_v2      0.820000   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   TRADES_v2           NaN   
eva02_base_patch14_224.mim_in22k                TRADES_v2           NaN   
vit_base_patch16_clip_224.laion2b               TRADES_v2      0.829000   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   CLASSIC_AT          NaN   
vit_base_patch16_224.augreg_in1k                TRADES_v2      0.748000   
robust_convnext_base                            TRADES_v2           NaN   
vit_base_patch16_224.augreg_in1k                CLASSIC_AT     0.756000   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k TRADES_v2           NaN   
robust_convnext_base                            CLASSIC_AT          NaN   
convnext_base.fb_in22k_ft_in1k                  TRADES_v2   